# How to use image moderation

This Python script demonstrates how to use OpenAI's Moderation API to analyze image URLs for compliance with moderation policies. The example includes two key functions: one for checking individual image URLs (`check_image_moderation`) and another for batch processing multiple URLs (`batch_process_image_urls`). The script not only identifies whether an image is approved for upload but also provides detailed feedback on flagged categories, helping users understand why an image may not meet the criteria. Additionally, the batch processing results are displayed in a neatly formatted table for better readability, making this tool practical for reviewing multiple images efficiently. For more information about using the Moderation API for images, visit the [OpenAI Moderation Guide](https://platform.openai.com/docs/guides/moderation?example=images).

To try out this example, first install the dependencies `openai`, `pandas`, `tabulate` and then modify the `image_urls` as necessary.

This API can be incredibly useful in a variety of real-world scenarios where content moderation is critical. For instance, social media platforms can use it to automatically screen user-uploaded images for harmful or inappropriate content, ensuring compliance with community guidelines. E-commerce platforms could utilize it to prevent the listing of prohibited or counterfeit items. Additionally, educational platforms and online forums can leverage this tool to maintain a safe and respectful environment by filtering out offensive imagery. Organizations handling large volumes of user-generated content, such as news agencies or content-sharing platforms, can use the batch processing feature to efficiently review multiple images at once, saving time while promoting ethical and responsible content dissemination. This solution not only aids in regulatory compliance but also helps foster trust and safety in digital communities.



In [6]:
from openai import OpenAI
import os
import pandas as pd

# Initialize OpenAI client
client = OpenAI()

def check_image_moderation(image_url):
    """
    Checks if an image URL passes OpenAI's moderation criteria.

    Parameters:
        image_url (str): The URL of the image to be checked.

    Returns:
        dict: A dictionary containing the URL, approval status, and reasons if not approved.
    """
    try:
        # Call the moderation API with the image URL
        response = client.moderations.create(
            model="omni-moderation-latest",
            input=[
                {
                    "type": "image_url",
                    "image_url": {
                        "url": image_url
                    }
                }
            ]
        )

        # Extract the moderation categories and their flags
        results = response.results[0]
        flagged_categories = vars(results.categories)
        flagged = results.flagged

        if not flagged:
            return {"url": image_url, "approved": True, "comments": "The image is good and can be uploaded."}
        else:
            reasons = [category.capitalize() for category, is_flagged in flagged_categories.items() if is_flagged]
            return {"url": image_url, "approved": False, "comments": ", ".join(reasons)}

    except Exception as e:
        return {"url": image_url, "approved": False, "comments": f"Error: {e}"}


def batch_process_image_urls(image_urls):
    """
    Processes a batch of image URLs and prints the results in a table.

    Parameters:
        image_urls (list): A list of image URLs to be checked.

    Returns:
        None
    """
    results = []

    # Check each URL in the batch
    for idx, url in enumerate(image_urls, start=1):
        print(f"Processing item {idx}/{len(image_urls)}: {url}")
        result = check_image_moderation(url)
        results.append({"Item": idx, **result})

    # Create a DataFrame for better visualization
    df = pd.DataFrame(results)
    print("\nBatch Processing Results:")
    print(df.to_markdown(index=False))

# Example batch of image URLs to check
image_urls = [
    "https://assets.editorial.aetnd.com/uploads/2009/10/world-war-one-gettyimages-90007631.jpg",
    "https://whc.unesco.org/uploads/thumbs/site_0252_0008-360-360-20250108121530.jpg"
]

batch_process_image_urls(image_urls)


Processing item 1/2: https://assets.editorial.aetnd.com/uploads/2009/10/world-war-one-gettyimages-90007631.jpg
Processing item 2/2: https://whc.unesco.org/uploads/thumbs/site_0252_0008-360-360-20250108121530.jpg

Batch Processing Results:
|   Item | url                                                                                       | approved   | comments                               |
|-------:|:------------------------------------------------------------------------------------------|:-----------|:---------------------------------------|
|      1 | https://assets.editorial.aetnd.com/uploads/2009/10/world-war-one-gettyimages-90007631.jpg | False      | Violence                               |
|      2 | https://whc.unesco.org/uploads/thumbs/site_0252_0008-360-360-20250108121530.jpg           | True       | The image is good and can be uploaded. |
